# PyTorch를 이용한 Iris 분류 모델

이 노트북은 PyTorch를 사용하여 Iris 데이터셋을 분류하는 신경망 모델을 구현합니다.

## 학습 목표
- PyTorch 기본 구조 이해
- 다층 퍼셉트론(MLP) 구현
- 데이터 전처리 및 DataLoader 사용
- 모델 학습 및 평가 과정 이해

## 1. 라이브러리 임포트

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

## 2. 데이터 로딩 및 탐색

In [ ]:
# Iris 데이터셋 로딩
iris = load_iris()
X = iris.data
y = iris.target

# 데이터셋 정보 출력
print(f"특성 개수: {X.shape[1]}")
print(f"샘플 개수: {X.shape[0]}")
print(f"클래스 개수: {len(np.unique(y))}")
print(f"\n특성 이름: {iris.feature_names}")
print(f"클래스 이름: {iris.target_names}")

# 클래스별 데이터 분포
unique, counts = np.unique(y, return_counts=True)
print(f"\n클래스별 데이터 분포:")
for i, (cls, count) in enumerate(zip(iris.target_names, counts)):
    print(f"  {cls}: {count}개")

## 3. 데이터 전처리

In [ ]:
# 데이터 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("표준화 전:")
print(f"평균: {np.round(X.mean(axis=0), 3)}")
print(f"표준편차: {np.round(X.std(axis=0), 3)}")

print("\n표준화 후:")
print(f"평균: {np.round(X_scaled.mean(axis=0), 3)}")
print(f"표준편차: {np.round(X_scaled.std(axis=0), 3)}")

# 특성별 상세 통계
print("\n특성별 상세 통계 (표준화 전):")
for i, feature_name in enumerate(iris.feature_names):
    print(f"  {feature_name}: 평균={X[:, i].mean():.2f}, 표준편차={X[:, i].std():.2f}")
    
print("\n특성별 상세 통계 (표준화 후):")
for i, feature_name in enumerate(iris.feature_names):
    print(f"  {feature_name}: 평균={X_scaled[:, i].mean():.2f}, 표준편차={X_scaled[:, i].std():.2f}")

In [ ]:
# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"학습 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")

# 분할 후 클래스 분포 확인
print(f"\n학습 데이터 클래스 분포: {np.bincount(y_train)}")
print(f"테스트 데이터 클래스 분포: {np.bincount(y_test)}")

## 4. PyTorch 텐서 변환 및 DataLoader 생성

In [ ]:
# NumPy 배열을 PyTorch 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

print(f"학습 특성 텐서 크기: {X_train_tensor.shape}")
print(f"학습 라벨 텐서 크기: {y_train_tensor.shape}")
print(f"특성 데이터 타입: {X_train_tensor.dtype}")
print(f"라벨 데이터 타입: {y_train_tensor.dtype}")

In [ ]:
# Dataset과 DataLoader 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"배치 크기: {batch_size}")
print(f"학습 배치 개수: {len(train_loader)}")
print(f"테스트 배치 개수: {len(test_loader)}")

# 첫 번째 배치 확인
for batch_features, batch_labels in train_loader:
    print(f"\n첫 번째 배치 특성 크기: {batch_features.shape}")
    print(f"첫 번째 배치 라벨 크기: {batch_labels.shape}")
    break

## 5. 신경망 모델 정의

In [ ]:
class IrisClassifier(nn.Module):
    """
    Iris 분류를 위한 다층 퍼셉트론 모델
    
    구조:
    - 입력층: 4개 특성
    - 은닉층1: 16개 뉴런 (ReLU 활성화)
    - 은닉층2: 8개 뉴런 (ReLU 활성화)
    - 출력층: 3개 클래스
    """
    
    def __init__(self, input_size=4, hidden1_size=16, hidden2_size=8, num_classes=3):
        super(IrisClassifier, self).__init__()
        
        # 레이어 정의
        self.fc1 = nn.Linear(input_size, hidden1_size)  # 입력 → 첫 번째 은닉층
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)  # 첫 번째 → 두 번째 은닉층
        self.fc3 = nn.Linear(hidden2_size, num_classes)  # 두 번째 은닉층 → 출력층
        
        # 활성화 함수
        self.relu = nn.ReLU()
        
        # 드롭아웃 (과적합 방지)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        """
        순전파 함수
        """
        # 첫 번째 은닉층
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # 두 번째 은닉층
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # 출력층 (softmax는 CrossEntropyLoss에서 자동 적용)
        x = self.fc3(x)
        
        return x

# 모델 인스턴스 생성
model = IrisClassifier()
print("모델 구조:")
print(model)

# 모델 파라미터 개수 계산
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n총 파라미터 개수: {total_params:,}")
print(f"학습 가능한 파라미터 개수: {trainable_params:,}")

## 6. 손실함수와 옵티마이저 설정

In [ ]:
# 손실함수: 다중 클래스 분류를 위한 CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

# 옵티마이저: Adam 옵티마이저
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

# 학습률 스케줄러 (선택사항)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

print(f"손실함수: {criterion}")
print(f"옵티마이저: {optimizer}")
print(f"초기 학습률: {optimizer.param_groups[0]['lr']}")

## 7. 모델 학습

In [ ]:
def train_model(model, train_loader, criterion, optimizer, epochs=100, print_every=10):
    """
    모델 학습 함수
    
    Args:
        model: 학습할 모델
        train_loader: 학습 데이터 로더
        criterion: 손실함수
        optimizer: 옵티마이저
        epochs: 학습 에포크 수
        print_every: 출력 주기
    
    Returns:
        loss_history: 손실값 히스토리
    """
    model.train()  # 모델을 학습 모드로 설정
    loss_history = []
    
    print("학습 시작...")
    print("-" * 50)
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        num_batches = 0
        
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            # 그래디언트 초기화
            optimizer.zero_grad()
            
            # 순전파
            outputs = model(inputs)
            
            # 손실 계산
            loss = criterion(outputs, labels)
            
            # 역전파
            loss.backward()
            
            # 가중치 업데이트
            optimizer.step()
            
            epoch_loss += loss.item()
            num_batches += 1
        
        # 에포크 평균 손실
        avg_loss = epoch_loss / num_batches
        loss_history.append(avg_loss)
        
        # 학습률 스케줄러 업데이트
        scheduler.step()
        
        # 진행 상황 출력
        if (epoch + 1) % print_every == 0 or epoch == 0:
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Epoch [{epoch+1:3d}/{epochs}] | Loss: {avg_loss:.4f} | LR: {current_lr:.6f}")
    
    print("-" * 50)
    print("학습 완료!")
    
    return loss_history

# 모델 학습 실행
epochs = 100
loss_history = train_model(model, train_loader, criterion, optimizer, epochs, print_every=20)

## 8. 학습 곡선 시각화

In [ ]:
# 학습 곡선 그래프
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(loss_history) + 1), loss_history, 'b-', linewidth=2)
plt.title('학습 곡선 (Loss)', fontsize=16, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"최종 손실값: {loss_history[-1]:.4f}")
print(f"최소 손실값: {min(loss_history):.4f} (Epoch {loss_history.index(min(loss_history)) + 1})")

## 9. 모델 평가

In [ ]:
def evaluate_model(model, data_loader, dataset_name="데이터셋"):
    """
    모델 평가 함수
    
    Args:
        model: 평가할 모델
        data_loader: 평가 데이터 로더
        dataset_name: 데이터셋 이름
    
    Returns:
        accuracy: 정확도
        predictions: 예측값
        true_labels: 실제값
    """
    model.eval()  # 모델을 평가 모드로 설정
    
    all_predictions = []
    all_labels = []
    correct = 0
    total = 0
    
    with torch.no_grad():  # 그래디언트 계산 비활성화
        for inputs, labels in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)  # 가장 높은 확률의 클래스 선택
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    print(f"{dataset_name} 정확도: {accuracy:.2f}% ({correct}/{total})")
    
    return accuracy, all_predictions, all_labels

# 학습 데이터와 테스트 데이터 평가
print("=" * 50)
print("모델 평가 결과")
print("=" * 50)

train_acc, train_pred, train_true = evaluate_model(model, train_loader, "학습")
test_acc, test_pred, test_true = evaluate_model(model, test_loader, "테스트")

print(f"\n과적합 정도: {train_acc - test_acc:.2f}%")

## 10. 상세 분류 성능 분석

In [ ]:
# 분류 보고서
print("테스트 데이터 분류 보고서:")
print("=" * 50)
print(classification_report(test_true, test_pred, target_names=iris.target_names))

In [ ]:
# 혼동 행렬 시각화
plt.figure(figsize=(8, 6))
cm = confusion_matrix(test_true, test_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=iris.target_names, 
            yticklabels=iris.target_names)
plt.title('혼동 행렬 (Confusion Matrix)', fontsize=16, fontweight='bold')
plt.xlabel('예측 클래스', fontsize=12)
plt.ylabel('실제 클래스', fontsize=12)
plt.tight_layout()
plt.show()

# 클래스별 정확도 계산
class_accuracy = cm.diagonal() / cm.sum(axis=1) * 100
print("\n클래스별 정확도:")
for i, (cls_name, acc) in enumerate(zip(iris.target_names, class_accuracy)):
    print(f"  {cls_name}: {acc:.1f}%")

## 11. 예측 확률 분석

In [ ]:
# 테스트 데이터의 예측 확률 분석
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_probs = torch.softmax(test_outputs, dim=1)

# 예측 확률 히스토그램
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for i, class_name in enumerate(iris.target_names):
    class_probs = test_probs[:, i].numpy()
    axes[i].hist(class_probs, bins=20, alpha=0.7, color=f'C{i}')
    axes[i].set_title(f'{class_name} 클래스 예측 확률')
    axes[i].set_xlabel('확률')
    axes[i].set_ylabel('빈도')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 가장 확신 있는 예측과 가장 불확실한 예측
max_probs = torch.max(test_probs, dim=1)[0]
most_confident_idx = torch.argmax(max_probs)
least_confident_idx = torch.argmin(max_probs)

print(f"가장 확신 있는 예측:")
print(f"  샘플 인덱스: {most_confident_idx}")
print(f"  예측 클래스: {iris.target_names[test_pred[most_confident_idx]]}")
print(f"  확률: {max_probs[most_confident_idx]:.4f}")

print(f"\n가장 불확실한 예측:")
print(f"  샘플 인덱스: {least_confident_idx}")
print(f"  예측 클래스: {iris.target_names[test_pred[least_confident_idx]]}")
print(f"  확률: {max_probs[least_confident_idx]:.4f}")

## 12. 새로운 데이터 예측 테스트

In [ ]:
def predict_sample(model, scaler, sample_features, feature_names, class_names):
    """
    새로운 샘플에 대한 예측 함수
    """
    model.eval()
    
    # 입력 데이터 전처리
    sample_scaled = scaler.transform([sample_features])
    sample_tensor = torch.tensor(sample_scaled, dtype=torch.float32)
    
    with torch.no_grad():
        output = model(sample_tensor)
        probabilities = torch.softmax(output, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
    
    print(f"입력 특성:")
    for name, value in zip(feature_names, sample_features):
        print(f"  {name}: {value}")
    
    print(f"\n예측 결과: {class_names[predicted_class]}")
    print(f"\n클래스별 확률:")
    for i, (class_name, prob) in enumerate(zip(class_names, probabilities[0])):
        print(f"  {class_name}: {prob:.4f} ({prob*100:.1f}%)")

# 예시 샘플들로 테스트
test_samples = [
    [5.1, 3.5, 1.4, 0.2],  # setosa 같은 특성
    [6.2, 2.9, 4.3, 1.3],  # versicolor 같은 특성
    [7.3, 2.9, 6.3, 1.8]   # virginica 같은 특성
]

for i, sample in enumerate(test_samples, 1):
    print(f"\n{'='*50}")
    print(f"테스트 샘플 {i}")
    print(f"{'='*50}")
    predict_sample(model, scaler, sample, iris.feature_names, iris.target_names)

## 13. 모델 저장 및 불러오기

In [ ]:
# 모델 저장
import os
import pickle

# 저장 경로 설정 및 디렉토리 생성
data_dir = os.path.join('..', '..', 'data')
os.makedirs(data_dir, exist_ok=True)

model_save_path = os.path.join(data_dir, 'iris_classifier_model.pth')
scaler_save_path = os.path.join(data_dir, 'iris_scaler.pkl')

# PyTorch 모델 저장 (가중치만 저장)
torch.save(model.state_dict(), model_save_path)
print(f"모델이 저장되었습니다: {model_save_path}")

# 스케일러 저장
with open(scaler_save_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"스케일러가 저장되었습니다: {scaler_save_path}")

# 모델 불러오기 예시
print("\n모델 불러오기 테스트...")
new_model = IrisClassifier()
new_model.load_state_dict(torch.load(model_save_path))
new_model.eval()

# 불러온 모델로 테스트
with torch.no_grad():
    test_output = new_model(X_test_tensor[:5])  # 처음 5개 샘플 테스트
    _, predicted = torch.max(test_output, 1)
    print(f"불러온 모델 예측 결과: {predicted.numpy()}")
    print(f"실제 라벨: {y_test[:5]}")
    print("모델이 성공적으로 저장/불러오기 되었습니다!")

# 저장된 파일 크기 확인
model_size = os.path.getsize(model_save_path)
scaler_size = os.path.getsize(scaler_save_path)
print(f"\n파일 크기:")
print(f"  모델 파일: {model_size:,} bytes ({model_size/1024:.1f} KB)")
print(f"  스케일러 파일: {scaler_size:,} bytes ({scaler_size/1024:.1f} KB)")

## 14. 실습 요약

### 주요 학습 내용

1. **PyTorch 기본 구조**
   - `nn.Module`을 상속한 모델 클래스 정의
   - `forward()` 메서드를 통한 순전파 구현
   - 텐서 변환과 데이터 타입 관리

2. **데이터 처리 파이프라인**
   - StandardScaler를 이용한 특성 정규화
   - `TensorDataset`과 `DataLoader`를 이용한 배치 처리
   - 학습/테스트 데이터 분할

3. **모델 구조**
   - 다층 퍼셉트론 (MLP) 구조
   - ReLU 활성화 함수
   - Dropout을 이용한 과적합 방지

4. **학습 과정**
   - CrossEntropyLoss 손실함수
   - Adam 옵티마이저
   - 학습률 스케줄링

5. **평가 및 분석**
   - 정확도, 혼동행렬, 분류보고서
   - 예측 확률 분석
   - 모델 저장 및 불러오기

### 성능 요약
- 최종 테스트 정확도: {test_acc:.1f}%
- 모델 복잡도: {total_params:,}개 파라미터
- 학습 시간: {epochs} 에포크

### 다음 단계
- 더 복잡한 데이터셋으로 실습
- CNN, RNN 등 다른 신경망 구조 학습
- 하이퍼파라미터 튜닝 및 교차검증